In [1]:
import asyncio
from playwright.async_api import async_playwright
from urllib.parse import urljoin
from pyquery import PyQuery as pq
import time
import json
import re
from motor.motor_asyncio import AsyncIOMotorClient
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s: %(message)s')

In [7]:
BASE_URL = 'https://www.qzsc.cc/'
cat_name = {
    'xuanhuanxiaoshuo/': '玄幻',
    'xiuzhenxiaoshuo/': '修真',
    'dushixiaoshuo/': '都市',
    }

In [8]:
for key, value in cat_name.items():
    print(key, value)
    print(type(key), type(value))
    url = urljoin(BASE_URL, key)
    print(url)

xuanhuanxiaoshuo/ 玄幻
<class 'str'> <class 'str'>
https://www.qzsc.cc/xuanhuanxiaoshuo/
xiuzhenxiaoshuo/ 修真
<class 'str'> <class 'str'>
https://www.qzsc.cc/xiuzhenxiaoshuo/
dushixiaoshuo/ 都市
<class 'str'> <class 'str'>
https://www.qzsc.cc/dushixiaoshuo/


In [2]:
string = '作    者：雪月天歌'

In [4]:
re.search(r'者：(.*)', string).group(1)

'雪月天歌'

In [5]:
doc = pq(filename='xs.html')

In [6]:
author = re.search(r'者：(.*)', doc('#info p:contains(者：)').text()).group(1)
last_update = re.search(r'更新：(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})', doc('#info p:contains(更新：)').text()).group(1)
brief = doc('#intro p').text()

In [7]:
print(author)

作    者：笔落尘凡 状    态： 连载, 【加入书架】, 【直达底部】 最后更新：2024-02-05 18:42:45 最新章节： 第581章 ：山河失色！区区小劫！


In [8]:
doc('#info p:contains(者：)').text()

'作\xa0\xa0\xa0\xa0者：笔落尘凡'

In [9]:
re.search(r'者：(.*)', doc('#info p:contains(者：)').text()).group(1)

'笔落尘凡'

In [10]:
re.search(r'更新：(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})', doc('#info p:contains(更新：)').text()).group(1)

'2024-02-05 18:42:45'

In [13]:
brief = doc('#intro p').text()

In [18]:
BASE_URL = 'https://www.qzsc.cc/'

In [19]:
chapters = doc('.box_con #list a')
chapter_info = []
for chapter in chapters.items():
    url = urljoin(BASE_URL, chapter.attr('href'))
    title = chapter.text()
    chapter_info.append({
        'url': url,
        'title': title
    })
print(chapter_info)

https://www.qzsc.cc/qz/153/153591/75736616.html 第一章：古钟三响！洛氏之危！
<class 'str'> <class 'str'>
https://www.qzsc.cc/qz/153/153591/90802420.html 第二章：力不从心，自此封山！
<class 'str'> <class 'str'>
https://www.qzsc.cc/qz/153/153591/55950037.html 第三章：太虚三章！寻求生机！
<class 'str'> <class 'str'>
https://www.qzsc.cc/qz/153/153591/64744929.html 第四章：寻找新的资源点！孤星山脉！
<class 'str'> <class 'str'>
https://www.qzsc.cc/qz/153/153591/55237676.html 第五章 ：深入蛮荒！灵脉之心！
<class 'str'> <class 'str'>
https://www.qzsc.cc/qz/153/153591/400430.html 第六章：聚四方灵气！灵脉之心融合！
<class 'str'> <class 'str'>
https://www.qzsc.cc/qz/153/153591/54572705.html 第七章：炼气九重！灵脉之心！
<class 'str'> <class 'str'>
https://www.qzsc.cc/qz/153/153591/30790163.html 第八章：巨大的收获！人心难测！
<class 'str'> <class 'str'>
https://www.qzsc.cc/qz/153/153591/90704490.html 第九章：回青山郡，赵氏修士！
<class 'str'> <class 'str'>
https://www.qzsc.cc/qz/153/153591/50583298.html 第十章：危机暂缓，筑基丹！
<class 'str'> <class 'str'>
https://www.qzsc.cc/qz/153/153591/69915812.html 第十一章：家族任务，阵法之说！
<class 'str'> <class 